In [1]:
import pandas as pd
import xarray as xr

# Utils
from sheerwater_benchmarking.utils import start_remote, salient_secret, clip_region, get_region

# Salient functions
from salientsdk.skill import _crps_core
import salientsdk as sk

In [2]:
start_remote(remote_name='genevieve', remote_config='xlarge_cluster')

Output()

## Run tests on Salient evaluation period, for a specific variable and lead

In [3]:
start_time = '2015-01-01'
end_time = '2022-12-31'
variable = 'precip'
var = {"tmp2m": "temp", "precip": "precip"}[variable] # salient naming
metric = 'crps'
region = 'africa'
mask = None
lead = 'week3'
timescale = 'sub-seasonal'

## Pull both forecasts and gt directly from the bucket

In [4]:
filename = f'gs://sheerwater-datalake/salient-data/v9/africa/{var}_{timescale}/blend'
fcst_ds = xr.open_zarr(filename)
fcst_ds = fcst_ds['vals'].to_dataset()
fcst_ds = fcst_ds.rename(vals=variable)
fcst_ds = fcst_ds.sel(forecast_date=slice(start_time, end_time))
fcst_ds

<xarray.Dataset> Size: 47GB
Dimensions:        (forecast_date: 1088, lat: 300, lead: 5, lon: 316,
                    quantiles: 23)
Coordinates:
  * forecast_date  (forecast_date) datetime64[ns] 9kB 2015-01-01 ... 2022-12-28
  * lat            (lat) float32 1kB -35.88 -35.62 -35.38 ... 38.38 38.62 38.88
  * lead           (lead) int64 40B 1 2 3 4 5
  * lon            (lon) float32 1kB -18.88 -18.62 -18.38 ... 59.38 59.62 59.88
  * quantiles      (quantiles) float64 184B 0.01 0.025 0.05 ... 0.95 0.975 0.99
Data variables:
    precip         (forecast_date, lead, lat, lon, quantiles) float32 47GB dask.array<chunksize=(1, 5, 300, 316, 23), meta=np.ndarray>

In [5]:
filename = f'gs://sheerwater-datalake/salient-data/v9/africa/{var}_{timescale}/truth'
gt_ds = xr.open_zarr(filename)
gt_ds = gt_ds['vals_actual'].to_dataset()
gt_ds = gt_ds.rename(vals_actual=variable)
gt_ds = gt_ds.sel(forecast_date=fcst_ds.forecast_date)
gt_ds

<xarray.Dataset> Size: 2GB
Dimensions:        (forecast_date: 1088, lat: 300, lead: 5, lon: 316)
Coordinates:
  * forecast_date  (forecast_date) datetime64[ns] 9kB 2015-01-01 ... 2022-12-28
  * lat            (lat) float32 1kB -35.88 -35.62 -35.38 ... 38.38 38.62 38.88
  * lead           (lead) int64 40B 1 2 3 4 5
  * lon            (lon) float32 1kB -18.88 -18.62 -18.38 ... 59.38 59.62 59.88
Data variables:
    precip         (forecast_date, lead, lat, lon) float32 2GB dask.array<chunksize=(1, 5, 300, 316), meta=np.ndarray>

In [6]:
point_lon, point_lat = (25.125, 0.125)

met_ds = _crps_core(observations=gt_ds, forecasts=fcst_ds, qnt_dim='quantiles')
met_ds = met_ds.mean('forecast_date')
# met = met_ds.sel(lon=point_lon, lat=point_lat)[metric].compute()
met_ds = clip_region(met_ds, 'kenya')
met = met_ds.mean(['lat', 'lon'])[metric].compute()

In [7]:
print(met.to_pandas())

lead
1    0.567062
2    0.735259
3    0.837947
4    0.885962
5    0.912672
Name: crps, dtype: float64


In [10]:
import geopandas as gpd 
from shapely.geometry import MultiPolygon, Polygon

lons, lats, mask = get_region('kenya')

# Get an african bounding geojson, the same as is used to clip above, and prep for Salient loc
# Add a dummy column for grouping
mask['group'] = 1

# Dissolve all countries into a single outline
full_mask = mask.dissolve(by='group')
full_geom = full_mask.geometry[1]

# Initialize an empty list to hold all coordinates
all_coordinates = [list(poly.exterior.coords) for poly in full_geom.geoms]
# Flatten
all_coordinates = [coord for poly_coords in all_coordinates for coord in poly_coords]

In [11]:
all_coordinates_swap = [(x[1], x[0]) for x in all_coordinates]

In [14]:
username, password = salient_secret()
sk.login(username, password)
loc = sk.Location(shapefile=sk.upload_shapefile(
    coords=all_coordinates,
    geoname="africa_region_swap", # the full African continent
    force=True)) 

In [15]:
# username, password = salient_secret()
# sk.login(username, password)
# loc = sk.Location(lat=point_lat, lon=point_lon) # Single point eval (lon, lat)

# The variable that we'll be evaluating.
fld = "vals"
timescale = "sub-seasonal"
ref_model = "clim"  # Works across all timescale values.

skill_summ = pd.read_csv(
    sk.hindcast_summary(
        loc=loc,
        metric="crps",
        variable=var,
        timescale=timescale,
        reference=ref_model,
        split_set="test",
        force=True
    )
)
print(var, timescale, ref_model)
print(skill_summ.drop(columns="Reference Model"))

precip sub-seasonal clim
     Lead  Reference CRPS  Salient CRPS  Salient CRPS Skill Score (%)
0  Week 1            0.96          0.58                          41.4
1  Week 2            0.96          0.75                          22.5
2  Week 3            0.96          0.86                          11.3
3  Week 4            0.97          0.91                           6.9
4  Week 5            0.97          0.94                           4.2


In [16]:
print(var)

precip
